# Бинарная классификация объектов на основе GAP

Реализован простой подход к классической задаче с использованием экстракции признаков на основе предобученных моделей, доступных во фреймворке tensorflow. После получения фич, они преобразуются в векторные признаки для каждой картинки. Далее мы строим простую сверточную нейронную сеть, делаем небольшой тюнинг гипермараметров и обучаем модель, которая выдаёт вероятности принадлежности классу от 0 до 1. Чем ближе к 1, тем более вероятно, что это собака В результате такой подход даёт высокую точность бинарного распредления картинок на валидационных данных и хорошо показывает себя на тестовых данных.
В конечном счете, реализована кнопка для загрузки любого изображения с локального компьютера, которое будет классифицировано кошкой или собакой.

In [ ]:
# установка некоторых библиотек
!pip install ipywidgets pydot graphviz

In [ ]:
# импорт библиотек
import zipfile
import os
import shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import *
import h5py
from PIL import Image
from sklearn.utils import shuffle
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import ipywidgets as widgets
from IPython.display import display
import io
np.random.seed(2041)

In [ ]:
# Распакуем данные в текущий рабочий каталог

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as zip_ref:
    zip_ref.extractall('train')
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as zip_ref:
    zip_ref.extractall('test')

In [ ]:
# Для удобства разобьем на подкаталоги папку train

train_filenames = os.listdir('train/train')
train_cat = [filename for filename in train_filenames if filename.startswith('cat')]
train_dog = [filename for filename in train_filenames if filename.startswith('dog')]
os.makedirs('train/cat', exist_ok=True)
os.makedirs('train/dog', exist_ok=True)

for filename in train_cat:
    src_path = os.path.join('train/train', filename)
    dest_path = os.path.join('train/cat', filename)
    shutil.move(src_path, dest_path)

for filename in train_dog:
    src_path = os.path.join('train/train', filename)
    dest_path = os.path.join('train/dog', filename)
    shutil.move(src_path, dest_path)
    
# Удаляем лишнюю папку
!cd train && rm -r train

## Определим рабочие функции

In [ ]:
def write_gap(MODEL, image_size, lambda_func=None):
    """
    Функция для вычисления и сохранения глобальных усредненных признаков (GAP) с использованием предобученной модели.

    Args:
        MODEL (function): Функция, представляющая предобученную модель.
        image_size (tuple): Размер входных изображений (высота, ширина) в пикселях.
        lambda_func (function, optional): Функция предобработки изображения (например, preprocess_input). По умолчанию None.
    """
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{}".format(i) for i in range(num_gpus)])
    
    with strategy.scope():
        base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
        model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train", image_size, shuffle=False, batch_size=16)
    test_generator = gen.flow_from_directory("test", image_size, shuffle=False, batch_size=16, class_mode=None)
    train = model.predict(train_generator, steps=train_generator.n)
    test = model.predict(test_generator, steps=test_generator.n)
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as h:
        model.save(h, "model")
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
    
def create_model(optimizer='adam', activation='sigmoid', dropout_rate=0.5):
    """
    Функция для создания модели нейронной сети
    """
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def tune_hyperparameters(X_train, y_train):
    """
    Функция для подбора гиперпараметров
    """
    model = KerasClassifier(build_fn=create_model, verbose=0)
    param_grid = {
        'optimizer': ['adam', 'rmsprop', 'sgd', 'adadelta', 'adagrad', 'ftrl', 'nadam'],
        'dropout_rate': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    }
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
    grid_result = grid.fit(X_train, y_train)
    print("Лучшие параметры: ", grid_result.best_params_)
    return grid_result.best_params_

def determine_class(label):
    """
    Определяет класс на основе значения метки (вероятности).

    Args:
        label (float): Метка (вероятность) для определения класса.

    Returns:
        str: Название класса ('это пес', 'это кит' или 'невероятно!').
    """
    if label >= dog_threshold:
        return "это пес"
    elif label <= cat_threshold:
        return "это кит"
    else:
        return "невероятное нечто!"
    
def w_gap_img(image_path, MODEL, image_size, lambda_func = None):
    """
    Функция для вычисления глобальных усредненных признаков (GAP) для одной картинки с использованием предобученной модели.

    Args:
        image_path (str): Путь к изображению.
        MODEL (function): Функция, представляющая предобученную модель.
        image_size (tuple): Размер входных изображений (высота, ширина) в пикселях.

    Returns:
        features (numpy.ndarray): Массив с извлеченными признаками.
    """
    width = image_size[0]
    height = image_size[1]
    img = Image.open(image_path)
    img = img.resize((width, height))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    input_tensor = Input(shape=(height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(inputs=input_tensor, outputs=GlobalAveragePooling2D()(base_model.output))
    features = model.predict(img_array)
    return features

def predict_label_img(image_path):
    """
    Функция для предсказания метки для одной картинки
    """
    ResNet50_f = w_gap_img(image_path, ResNet50, (224, 224))
    InceptionV3_f = w_gap_img(image_path, InceptionV3, (299, 299), inception_v3.preprocess_input)
    Xception_f = w_gap_img(image_path, Xception, (299, 299), xception.preprocess_input)
    combined_features = np.concatenate([ResNet50_f, InceptionV3_f, Xception_f], axis=1)
    prediction = best_model.predict(combined_features)
    return determine_class(prediction)


def handle_upload(change):
    """
    Обработчик события нажатия кнопки загрузки изображения.

    Args:
        change (dict): Информация о событии изменения (загрузки).
        
    """
    uploaded_filename = next(iter(upload_button.value))
    content = upload_button.value[uploaded_filename]['content']
    global img
    
    with output_image:
        output_image.clear_output()
        img = Image.open(io.BytesIO(content))
        display(img)
    
    temp_dir = "temp_images"
    os.makedirs(temp_dir, exist_ok=True)
    temp_image_path = os.path.join(temp_dir, uploaded_filename)
    img.save(temp_image_path)
    predicted_label = predict_label_img(temp_image_path)
    with output_image:
        display(predicted_label)
        
def load_gap_data(filenames):
    """
    Загружает данные из списка файлов.

    Args:
        filenames (list): Список имен файлов, которые содержат данные для загрузки и предобработки.

    Returns:
        X_data (numpy.ndarray): Массив с вектором признаков.
        y_data (numpy.ndarray): Массив с метками классов.
    """
    X_data = []
    X_test = []
    y_data = None
    for filename in filenames:
        with h5py.File(filename, 'r') as h:
            X_data.append(np.array(h['train']))
            X_test.append(np.array(h['test']))
            if y_data is None:
                y_data = np.array(h['label'])
    X_data = np.concatenate(X_data, axis=1)
    X_test = np.concatenate(X_test, axis=1)
    X_data, y_data = shuffle(X_data, y_data)
    return X_data, y_data, X_test

## Feature extraction (Global Average Pooling)

In [ ]:
# Cколько GPU доступно в системе
num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)
data_files = ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]
X_train, y_train, X_test = load_gap_data(data_files)

## Обучение модели CNN

In [ ]:
# Определение лучших гиперпараметров
best_params = tune_hyperparameters(X_train, y_train)

# Создание модели с лучшими параметрами
best_model = create_model(optimizer=best_params['optimizer'], dropout_rate=best_params['dropout_rate'])

# Обучение модели
history = best_model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, verbose=1)

# Получение истории обучения
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# График потерь на обучающем и валидационном наборе данных
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Epoch')

# График точности на обучающем и валидационном наборе данных
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Train Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy vs. Epoch')

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация модели
plot_model(best_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Матрица ошибок на валидационном наборе данных

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_true_val = y_train[-int(len(y_train) * 0.2):]
y_pred_val = best_model.predict(X_train[-int(len(y_train) * 0.2):])
y_pred_val = (y_pred_val > 0.95)
confusion = confusion_matrix(y_true_val, y_pred_val)

plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print("Где коты - 0, собаки - 1")


## Сохранение предсказаний

In [ ]:
# Сохраним модель
best_model.save('best_model.h5')

# Запишем предсказания
y_pred = best_model.predict(X_test, verbose=1)
# y_pred = y_pred.clip(min=0.005, max=0.995) # можно вкл 
df = pd.read_csv("/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv")
image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)
for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.at[index-1, 'label'] = y_pred[i]

# Порог значений вероятностей (задается эмпирически. Если строго, то от 0.05 до 0.95)
dog_threshold = 0.95
cat_threshold = 0.05
        
df['class'] = df['label'].apply(determine_class)
df.to_csv('pred.csv', index=None)

## Демонстрация модели на 1 изображении

In [ ]:
# Для загрузки изображения
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
display(upload_button)

# Для отображения загруженного изображения
output_image = widgets.Output()
display(output_image)

# Переменная для хранения загруженного изображения
img = None

# Привязываем обработчик к кнопке
upload_button.observe(handle_upload, names='value')